In [15]:
# !pip install xlsxwriter
import pandas as pd
import numpy as np
import glob
from datetime import datetime as dt
from random import randint
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_rows = 22
pd.set_option('display.max_columns', None)


In [16]:
# from google.colab import drive
# drive.mount('/content/drive')

In [17]:
def autosize_excel_columns(worksheet, df):
  for idx, col in enumerate(df):
    series = df[col]
    max_len = max((
      series.astype(str).map(len).max(),
      len(str(series.name))))
    worksheet.set_column(idx, idx, max_len)

In [18]:
def create_report(path):
    xlsx_list = sorted(glob.glob(f'{path}*.xlsx'))
    output = pd.DataFrame()
    for name in xlsx_list:
        df = pd.read_excel(name, sheet_name = 'сводная таблица',
                           index_col = [0, 1],
                           usecols = [0, 1, 2, 3, 4])
        df = df.fillna('0').drop(index='Все')
        sums = df.groupby(level = 0).sum().drop(columns='Процент неликвидов')
        df.drop(columns = ['Всего новинок', 'Всего неликвидов'], inplace=True)
        sums['ИТОГ % НЕЛИКВИДОВ'] = sums['Всего неликвидов'] / sums['Всего новинок']
        df['Процент неликвидов'] /= 100
        sums.rename(columns={'Всего новинок':'ИТОГ ШТ. НОВИНОК',
                             'Всего неликвидов':'ИТОГ ШТ. НЕЛИКВИДОВ'}, inplace=True)
        sums['ИТОГ ШТ. НОВИНОК'] = sums['ИТОГ ШТ. НОВИНОК'].astype(str) + 'шт.'
        sums['ИТОГ ШТ. НЕЛИКВИДОВ'] = sums['ИТОГ ШТ. НЕЛИКВИДОВ'].astype(str) + 'шт.'
        sums = sums.stack().to_frame(name='Процент неликвидов')
        sums['dummy'] = 1
        period = pd.concat([df, sums])
        period.rename(columns={'Процент неликвидов':name[-15:-8]}, inplace=True)
        output = pd.concat([output, period], axis=1)
    output['sorter'] = output['dummy'].sum(axis=1)
    output = output.sort_values(by=['Редакция', 'sorter']).drop(columns=['dummy', 'sorter'])
    return output.reset_index()

In [23]:
# Write pivot table to file
output = create_report('input/ЭКСМО/')
# output = create_report('drive/MyDrive/Colab Notebooks/input/ЭКСМО/')
writer = pd.ExcelWriter(f'out.xlsx', engine='xlsxwriter') 
output.to_excel(writer, index=False, sheet_name='сводная таблица', na_rep='')

# Autosize
workbook = writer.book
worksheet = writer.sheets['сводная таблица']
autosize_excel_columns(worksheet, output)

# Get graph data
graph1_data = output.loc[lambda df: df['Подразделение'].isin(['ИТОГ % НЕЛИКВИДОВ']), :]
graph1_data = graph1_data.loc[lambda df: df['Редакция'].isin(['100 - Редакция "Комильфо"',
                                                              '101 - Редакция № 1',
                                                              '102 - Редакция № 2',
                                                              '120 - Редакция № 5']), :]
# graph1_data = graph1_data.loc[lambda df: df['Редакция'].isin(['507 Департамент Планета Детства',
#                                                 '509 Департамент Художественной литературы',
#                                                 '510 Департамент Межиздат',
#                                                 '511 Департамент Прикладная литература']), :]

# cell formating

bald_outline = workbook.add_format({'bold': True,
                                    'border': 1})
bald_outline_color = workbook.add_format({'bold': True,
                                          'border': 1,
                                          'bg_color': 'orange'})




chart_red = workbook.add_chart({'type': 'line'})

(max_row, max_col) = graph1_data.shape

for i in graph1_data.index:
    worksheet.set_row(i, None, bald_outline_color)
    chart_red.add_series({
        'name': ['сводная таблица', i+1, 0, i+1, 0],
        'values': ['сводная таблица', i+1, 2, i+1, max_col-1],
        'marker': {'type': 'diamond', 'size': 5, 'fill': {'color': 'black'}},
        'categories' : ['сводная таблица', 0, 2, 0, max_col-1]})
chart_red.set_legend({'position':'bottom',
                      'font': {'size': 9}})

worksheet = workbook.add_chartsheet('Редакции')
worksheet.set_chart(chart_red)
writer.close()

0

0

0

0